In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
plt.style.use(['science', 'notebook'])
from openpyxl import load_workbook
import imageio
from scipy.interpolate import interp1d, interp2d

# Question 1

Let $p$ represent the photon beam and $e$ the electron beam. The main information is

* $D_{Q_p}' MU_p = 0.3D_Q$
* $D_{Q_e}' MU_e = 0.7D_Q$

These are given by

$$D_{Q_p}' = OAR(d_{Q_p}, x_{Q_p})S_c(r_c)S_p(r_d) \cdot 1~\text{cGy/MU}$$
$$D_{Q_e}' = PDD(r_{\text{insert}}, d_{Q_e}) \left( \frac{SSD_{eff}+d_m}{SSD_{eff}+d_m+g} \right)^2 S_e(r_{\text{insert}},r_a) \cdot CF_{obl}(45^{\circ}, d_{Q_e}) \cdot  1~\text{cGy/MU}$$

We use the square root rule to obtain $S_e(r_i, r_a)$, $SSD_{eff}$, $d_m$, and $PDD$ for the electron beam since the insert size is rectangular

* **Note**: For the photon, the equivalent field size at the surface is the equivalent of a 15x6.5$~\text{cm}^2$ field, which is $9$cm. 

* **Note**: Field sizes are adjusted to be at dose point. Off axis points are adjusted to be in the plane of the isocenter

* **Note**: The obliquity angle of the electron beam is $45^{\circ}$; this follows from the $315^{\circ}$.

In [2]:
DQ = 4000
'''Photon'''
OAR = 1.022 #x=3.744, d=1.5
Sc = 0.998 #Xc=6.5, Yc=15
Sp = 0.997405 #rd = 9.135

'''Electron'''
P1 = 1; P2=0.99; P=np.sqrt(P1*P2) #d=2, table from exam
Se1 = 0.99*1; Se2 = 0.99*0.98; Se=np.sqrt(Se1*Se2) #applicator_f * insert_f
SSDe1 = 91.1; SSDe2 = 77.1; SSDe=np.sqrt(SSDe1*SSDe2) #sideofsquare=4.5 and 15
dm = np.sqrt(2*2.4) #not sure if this is a thing, but whatever
CF_obl = 1.06 # Rp=6, d=2, angle=45
g=10

Get the monitor units

In [3]:
DQp_rate = OAR*Sc*Sp
DQe_rate = P * ((SSDe+dm)/(SSDe+dm+g))**2 * Se * CF_obl
MUp = 0.3*DQ/DQp_rate
MUe = 0.7*DQ/DQe_rate
print(f'Monitor units photon: {MUp/20:.2f} MU /fraction ({MUp:.2f} MU total)')
print(f'Monitor units electron: {MUe/20:.2f} MU /fraction ({MUe:.2f} MU total)')

Monitor units photon: 58.98 MU /fraction (1179.58 MU total)
Monitor units electron: 168.77 MU /fraction (3375.46 MU total)


# Part B

For the photon beam we use

$$D_{L_p} = D_{Q_p} \frac{OAR(x_{L_p}, d_{L_p})}{OAR(x_{Q_p}, d_{Q_p})} \cdot PDD(d_{L_p},r_{d_{L_p}}) \cdot \left(\frac{SSD_0+d_{L_p}}{SSD_0+d_{L_p}-\Delta s}\right)^2 \left(TMR(d_{\text{lung,p}}, r_{d_{L_p}}\right)^{\rho_{\text{lung}}-1}$$

where $\Delta s$ is the difference in height of the surfaces (in this case, 0.3cm). The second last term corrects for the fact that the SSD is different for the raised surface. The final term is the Batho correction rule for the inhomogeneity. And for the electron beam we use

$$D_{L_e} = D_{Q_e} \frac{P(d_{\text{eff}}, r_{\text{insert}})}{P(d_{Q_e}, r_{\text{insert}})} \frac{CF_{obl}(45^{\circ}, d_{L_e})}{CF_{obl}(45^{\circ}, d_{Q_e})}$$

where $d_{\text{eff}} = d_{L_e} - d_{\text{lung,e}} + d_{\text{lung,e}} \cdot \text{CET}$, Thankfully everything is along the CAX, so no additional corrections need to be made.

In [4]:
xLp = 5.1*(100/103.1)
xQp = 3.8*(100/101.5)
rdLp = 9 * (103.1/100)
deff = 4 - 1.1 + 1.1 * 0.5
deff

3.45

Compute quantities

In [5]:
'''Photon'''
DQp = 0.3*DQ
OAR_L = 1.029 #x=4.95, d=3.1
OAR_Q = 1.022 #x=3.744, d=1.5
PDD = 0.9463 #d=3.1, r=9
CF1 = ((100+3.1)/(100+3.1-0.3))**2
CF2 = 1**(0.25-1) #d=1.3, rd=9.279 (note: tbles dont go below 1.5cm)
DLp = DQp * OAR_L / OAR_Q * PDD * CF1 * CF2

'''Electron'''
DQe = 0.7*DQ
PL = np.sqrt(0.896*0.95) # midterm table, d=3.4, use entries from each
PQ = np.sqrt(1*0.99)
CFL = 0.79 #d=4, angle=45
CFQ = 1.06 #d=2, angle=45
DLe = DQe * PL / PQ * CFL / CFQ

print(f'Dose to L from photon: {DLp/20:.2f}cGy/fraction ({DLp:.2f}cGy total)')
print(f'Dose to L from electron: {DLe/20:.2f}cGy/fraction ({DLe:.2f}cGy total)')

Dose to L from photon: 57.50cGy/fraction (1150.02cGy total)
Dose to L from electron: 96.75cGy/fraction (1934.98cGy total)


## Part C

For the dose to the heart, we correct for the depth and the off axis ratio using PDD and OAR, then correct for inhomogenity using the Batho correction factor:

$$D_H = \frac{PDD(d_H, r)}{PDD(d_Q, r)}\frac{OAR(d_{H}, x_H)}{OAR(d_Q, x_Q)} \left(\frac{TMR(d_{\text{stern}}+3.4, r_{d_H})}{TMR(3.4, r_{d_H})}\right)^{\rho_{\text{stern}}-1} D_Q$$

Get dose to heart

In [6]:
PDDnum = 85.81 #d=5.2, r=9
PDDden = 100 #d=1.5, r=9
OARnum = 1.005 #x=1.426, d=5.2
OARden = 1.022 #x=3.744, d=1.5
TMRnum = 0.942 #d=4.4, rd=9.468
TMRden = 0.969 #d=3.4, rd=9.468
DH = PDDnum/PDDden * OARnum/OARden * (TMRnum/TMRden)**(1.5-1) * DQp
print(f'Dose to heart: {DH/20:.2f}cGy/fraction ({DH:.2f}cGy total)')

Dose to heart: 49.92cGy/fraction (998.38cGy total)


## Part D

<p align="center">
  <img src="images/m1.png" alt="drawing" width="600" align="center"/>
</p>

# Question 2

Define all quantities

In [7]:
DAo1 = 1.194
DAw1 = 0.279
DAo2 = 0.481
DAw2 = 0.113
DBo1 = 0.481
DBw1 = 0.113
DBo2 = 1.194
DBw2 = 0.279
DCo1 = 0.983
DCw1 = 0.172
DCo2 = 0.983
DCw2 = 0.172

This question is simply a system of linear equations. Note that there are three equations (all doses equal to 214) but only two unknowns (the monitor units). While it seems that this system is overconstrained, the symmetry at points A and B don't yield two linear independent equations, so really there are only two unique equations (specified by A and C (or B and C)). This can be written using the following matrix:

$$\begin{bmatrix}\dot{D}_{Ao1}+\dot{D}_{Ao2} & \dot{D}_{Aw1}+\dot{D}_{Aw2}\\ \dot{D}_{Co1}+\dot{D}_{Co2} & \dot{D}_{Cw1}+\dot{D}_{Cw2} \end{bmatrix} \begin{bmatrix}MU_o \\ MU_w \end{bmatrix} = \begin{bmatrix}214 \\ 214 \end{bmatrix}$$

* First subscript specifies points (A/B/C)
* Second specifies open/wedged (o/w)
* Third specifies the beam (1/2)

We can invert this matrix to find the corresponding monitor units

In [8]:
M = np.array([[DAo1+DAo2, DAw1+DAw2],[DCo1+DCo2, DCw1+DCw2]])
b = np.array([214,214])
MUo, MUw = np.linalg.inv(M)@b

Lets check these are correct:

In [9]:
print(DAo1*MUo+DAw1*MUw+DAo2*MUo+DAw2*MUw)
print(DBo1*MUo+DBw1*MUw+DBo2*MUo+DBw2*MUw)
print(DCo1*MUo+DCw1*MUw+DCo2*MUo+DCw2*MUw)

214.0
214.0
214.0


The corresponding monitor units are:

In [10]:
print(f'Open monitor units: {MUo:.2f} MU')
print(f'Wedged monitor units: {MUw:.2f} MU')

Open monitor units: 52.82 MU
Wedged monitor units: 320.22 MU


## Part B

The ICRU dose point criteria are summarized as follows:

<p align="center">
  <img src="images/m2.png" alt="drawing" width="600" align="center"/>
</p>

The most difficult point to satisfy is likely point (4): we want to choose a region without a steep gradient. Observing the figure on the left, the region with the least gradient likely occurs directly in the center of the breast. This is shown below (red dot next to point C):

<p align="center">
  <img src="images/m3.png" alt="drawing" width="600" align="center"/>
</p>

## Part C

The purpose of the ICRU reference point is to give a **consistent** and **relevant** summary of the dose distribution given to a region of the body. By choosing this point in a region where the gradient is small, this ensures that small *pertubations* (slight relocations) of the point does not yield a significantly different result. In addition, the dose at the point gives a good estimate of what the dose throughout the target volume looks like (easy to give one number), without having to give a dose volume histogram (not convenient to give an entire plot). This makes it a meaningful parameter for reporting in 3D conformal radiation therapy.

## Part D

Judging by the figure below...

<p align="center">
  <img src="images/m2.png" alt="drawing" width="600" align="center"/>
</p>

... I would expect that the dose at the ICRU reference point would be **less** than that at points A and B and C. This is because there is typically more dose at the surface due to the patient surface contours that induce more radiation scattering.

* This can also be mathematically reasoned: the ICRU reference point should always be chosen at a location where $\nabla D=0$, and chosen in the center of the PTV. For curved regions, such as the one in the figure above, $\nabla^2 D>0$ in the center of the volume (as for typical beam configurations that have sufficient surface scatter, there is no reason the dose should *increase* as one goes towards the center of the PTV). It follows that the dose at the ICRU point is less than that at $A$, $B$, and $C$

## Part E

The dose at point C would likely be greater than 1% the calculated dose. As quoted from the lecture notes:

<p align="center">
  <img src="images/m4.png" alt="drawing" width="600" align="center"/>
</p>

The beam enters point C at highly oblique angles (based on the diagram shown), so the increased scatter near the surface will likely lead to a large dose at point C.

# Question 3

## Part A

In the diagram below

* Red represents the photon beam from Q1
* Dark blue (dashed) represents the electron beam from Q1
* Light blue (dotted) represents the parallel opposed pairs from Q2
* The distance 6.364 was obtained by $\sqrt{2} \times $ electron beam width (beam enters at $45^{\circ}$ to the patient, get $\sqrt{2}$ from this)

Note that all other relevant distances on the diagram can be derived from those shown on the figure.

<p align="center">
  <img src="images/m5.png" alt="drawing" width="600" align="center"/>
</p>

## Part B

Consider the following plot for electron isodose on a flat surface

<p align="center">
  <img src="images/m6.png" alt="drawing" width="200" align="center"/>
</p>

Note that as one gets deeper into the surface, the beam starts to widen. If the two beams entered the patient at the same angle (say, 315$^{\circ}$), and they were aligned at the surface such that there was no overlap, the two beams would eventually start to overlap deeper inside the patient.  By tilting the photon beams slightly outward (310$^{\circ}$) compared to the electron beam (315$^{\circ}$), the dose overlap due to the spreading of the electron beam becomes negated, even though they're still able to share a junction at the patient surface

## Part C

Consider the following diagram. Note that both beam pairs share the same isocenter, denoted as "parallel pair isocenter" in the diagram below. 

<p align="center">
  <img src="images/m7.png" alt="drawing" width="600" align="center"/>
</p>

In this scenario, $Y_1=0$ for the photon tangent beam pair. This permits a few new options with the anterior posterior pair:

* Assuming $Y_1$ is still points in the superior direction, the anterior-posterior pair could use $Y_2=0$ to match the junction of the two beam pairs. The extent of $Y_1$  will depend on the size of the supraclavical lymph nodes that need to be irradiated. 

Intuitively this makes sense; the two pairs of beams only differ by the rotation of the gantry; if we want them to have an overlapping junction along a horizontal line in the figure above, one pair must have $Y_1=0$ and the other must have $Y_2=0$ (since they share a common isocenter). 

A few precautions also have to be made due to the fact that there is also the other photon and electron beam. The value of $X_1$ for the two beam pairs **will need to be different**. While the tangential beam has $X_1=0$, the anterior posterior beam will need to have a value of $X_1$ that permits alignment the electron beam and the anterior-posterior beam at the surface of the patient. See the diagram below; note how the anterior-posterior beam needs to have beam on both the left ($X_1$ direction) and right ($X_2$ direction) of its CAX.

<p align="center">
  <img src="images/m8.png" alt="drawing" width="300" align="center"/>
</p>

The precautions are apparent in the diagram above: the intersection of the anterior-posterior beam and the tilted electron beam (predominantly inside the lung) need to bea accounted for. The addition of these two fields inside the lung could be quite damaging. This might involve using a lower energy photon beam for the anterior-posterior beam pair, to ensure that the photons don't travel as far inside the body. In addition, one may choose to have a smaller value of $X_1$ for the anterior-posterior pair. This would remove the alignment of the electron field and the anterior-posterior pair at the surface, but would lessen the dose on the side of the lung closest to the heart.

# Question 4

## Part A

<p align="center">
  <img src="images/m10.png" alt="drawing" width="600" align="center"/>
</p>

## Part B

If the heart colume is 208 cm$^{3}$, then 100 cm$^{3}$ corresponds to approximately 48.1% of the total volume.  The minimum dose to the hottest 48.1% of the volume is 775cGy, as seen on the dose-volume histogram.

## Part C

<p align="center">
  <img src="images/m11.png" alt="drawing" width="600" align="center"/>
</p>